In [1]:
import pandas as pd
import numpy as np
from feature_engineering import *

In [2]:
df = pd.read_csv('dsb-24-german-credit/german_credit_train.csv')

In [3]:
df = engineer_features(df)

In [4]:
df

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,CreditHistoryRisk,ExistingSavingsNumeric,EmploymentDurationYears,loan_to_income_proxy,debt_to_savings,loan_amount_x_credit_risk,employment_stability,residence_to_employment_ratio,dependents_per_age,prior_default_flag
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,2.0,300.0,0.5,629.664568,6.296666,1889.0,0.193548,5.999880,0.031250,0
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,2.0,50.0,2.5,230.998845,9.239998,462.0,1.666656,0.799997,0.027027,0
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,0.0,50.0,2.5,124.999375,4.999999,750.0,1.999984,1.199995,0.035714,1
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,2.0,50.0,8.0,1230.995897,73.859985,3693.0,3.428557,0.250000,0.031250,0
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,0.0,750.0,8.0,2078.326406,8.313333,18705.0,3.428557,0.375000,0.017544,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,greater_200,27,credits_paid_to_date,furniture,4650,less_100,1_to_4,3,male,none,...,2.0,50.0,2.5,1549.994833,92.999981,4650.0,1.111106,1.599994,0.025000,0
3995,0_to_200,11,prior_payments_delayed,furniture,250,greater_1000,4_to_7,3,male,none,...,0.0,1500.0,5.5,83.333056,0.166667,750.0,5.999935,0.545454,0.031250,1
3996,no_checking,32,outstanding_credit,appliances,6536,unknown,greater_7,5,male,co-applicant,...,NaN,NaN,8.0,1307.197386,NaN,NaN,2.999989,0.624999,0.037037,0
3997,0_to_200,38,outstanding_credit,other,1597,500_to_1000,greater_7,3,female,co-applicant,...,NaN,750.0,8.0,532.331559,2.129333,NaN,2.526308,0.375000,0.037037,0
